### 문제
- 축제 지역안에 포함된 집의 개수를 출력하는 문제 같은데, 축제는 매일 열리고 이미 방문한 집이였어도 다시 방문할 수 있는듯 하다.

### 접근
- 화성 지도처럼 시작선에 +1, 끝선에 -1을 하되 이번엔 그 값을 그대로 정답에 더하면 될거 같다.
- 왼쪽에서 오른쪽으로 스위핑, x축에 대해서 축제 지역을 먼저 업데이트하고 해당 x축에 속하는 집을 세면 될거같다.

In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class LazySegmentTree:
  def __init__(self, data, default=0, func=max):
    """initialize the lazy segment tree with data"""
    self._default = default
    self._func = func

    self._len = len(data)
    self._size = _size = 1 << (self._len - 1).bit_length()
    self._lazy = [0] * (2 * _size)

    self.data = [default] * (2 * _size)
    self.data[_size:_size + self._len] = data
    for i in reversed(range(_size)):
      self.data[i] = func(self.data[i + i], self.data[i + i + 1])

  def __len__(self):
    return self._len

  def _push(self, idx):
    """push query on idx to its children"""
    # Let the children know of the queries
    q, self._lazy[idx] = self._lazy[idx], 0

    self._lazy[2 * idx] += q
    self._lazy[2 * idx + 1] += q
    self.data[2 * idx] += q
    self.data[2 * idx + 1] += q

  def _update(self, idx):
    """updates the node idx to know of all queries applied to it via its ancestors"""
    for i in reversed(range(1, idx.bit_length())):
      self._push(idx >> i)

  def _build(self, idx):
    """make the changes to idx be known to its ancestors"""
    idx >>= 1
    while idx:
      self.data[idx] = self._func(self.data[2 * idx], self.data[2 * idx + 1]) + self._lazy[idx]
      idx >>= 1

  def add(self, start, stop, value):
    """lazily add value to [start, stop)"""
    start = start_copy = start + self._size
    stop = stop_copy = stop + self._size
    while start < stop:
      if start & 1:
        self._lazy[start] += value
        self.data[start] += value
        start += 1
      if stop & 1:
        stop -= 1
        self._lazy[stop] += value
        self.data[stop] += value
      start >>= 1
      stop >>= 1

    # Tell all nodes above of the updated area of the updates
    self._build(start_copy)
    self._build(stop_copy - 1)

  def query(self, start, stop, default=0):
    """func of data[start, stop)"""
    start += self._size
    stop += self._size

    # Apply all the lazily stored queries
    self._update(start)
    self._update(stop - 1)

    res = default
    while start < stop:
      if start & 1:
        res = self._func(res, self.data[start])
        start += 1
      if stop & 1:
        stop -= 1
        res = self._func(res, self.data[stop])
      start >>= 1
      stop >>= 1
    return res

def sol() :
  N, M = map(int, input().split())
  MAX = 10**5 + 1
  cnt = LazySegmentTree([0] * MAX, 0, lambda x, y: x + y)

  H = [] #Home
  for _ in range(N) :
    H.append(tuple(map(int, input().split())))
  H.sort(reverse=True)

  P = [] #Parade
  for _ in range(M) :
    x1, x2, y1, y2 = map(int, input().split())
    P.append((x1, y1, y2, 1))
    P.append((x2, y1, y2, -1))
  P.sort(reverse=True)

  answer = 0
  for X in range(MAX+1) :
    tmp = []
    while P and P[-1][0] == X:
      x, y1, y2, v = P.pop()
      if v == -1 : #끝지점은 나중에 처리
        tmp.append((x, y1, y2, v))
      else :
        assert(v == 1)
        cnt.add(y1, y2+1, v)
    
    while H and H[-1][0] == X :
      x, y = H.pop()
      answer += cnt.query(y, y+1)
    
    for x, y1, y2, v in tmp :
      assert(v == -1)
      cnt.add(y1, y2+1, v)

  sys.stdout.write(str(answer) + "\n")

for _ in range(int(input())): 
  sol()

### 풀이(자력솔)
- lazy의 원리는 모르는데 lazy를 써서 생각보다 빨리 풀었다
- 접근대로 풀되, x값에 대한 처리가 귀찮아서 그냥 range(0, 10**5)로 훑으면서 집이 있는지 없는지를 확인했다.
  - 이 때 축제의 끝선에 위치한 집도 계란을 던질 수 있기 때문에, 시작선 -> 집 확인 -> 끝선의 처리 순서로 진행했다.
- 그리고 10**5는 5만이 아니라 10만이다...
